In [10]:

from sentence_transformers import SentenceTransformer
import pandas as pd
import time
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
import os
import faiss
import numpy as np
import pickle

# Load the FAISS index
storage_directory = r'C:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\stored_data'
index = faiss.read_index(os.path.join(storage_directory, 'faiss_index.bin'))

# Load document texts
with open(os.path.join(storage_directory, 'documents.pkl'), 'rb') as f:
    documents = pickle.load(f)

# Load metadata
with open(os.path.join(storage_directory, 'metadata.pkl'), 'rb') as f:
    file_metadata = pickle.load(f)

# Ensure documents and metadata are loaded correctly
print(f"Loaded {len(documents)} documents.")
print(f"Loaded {len(file_metadata)} metadata entries.")

# Function to retrieve text and documents based on a query
def retrieve_text_and_doc(query, k=4):
    # Step 1: Compute the query embedding
    query_embedding = embedding_model.encode([query], convert_to_tensor=False)
    query_embedding = query_embedding / np.linalg.norm(query_embedding)

    # Check embedding size and FAISS index dimensionality
    assert query_embedding.shape[1] == index.d, f"Embedding size {query_embedding.shape[1]} does not match FAISS index dimensionality {index.d}"

    # Step 2: Perform the search in the FAISS index
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)

    # Step 3: Retrieve the corresponding documents and metadata
    retrieved_docs = []
    retrieved_metadata = []
    
    for i in indices[0]:
        if i < len(documents):
            retrieved_docs.append(documents[i])
            retrieved_metadata.append(file_metadata[i])
        else:
            print(f"Warning: Index {i} is out of range for documents. Total documents: {len(documents)}.")

    return retrieved_docs, retrieved_metadata, distances[0]

# Example usage
query = input("Enter your query: ")
retrieved_docs, retrieved_metadata, distances = retrieve_text_and_doc(query)

# Join and display retrieved text and metadata
retrieved_text = " ".join(retrieved_docs)
print(query)
for doc, meta, dist in zip(retrieved_docs, retrieved_metadata, distances):
    print(f"Metadata: {meta}, Distance: {dist}, Document: {doc[:100]}...")


Loaded 7 documents.
Loaded 7 metadata entries.
Revised+Synposis++-+6.14.23 (1)
Metadata: {'doc_number': 2, 'doc_name': 'Revised+Synposis++-+6.14.23 (1).pdf'}, Distance: 1.890243411064148, Document: Green underlined is add-in, Red strike through is deleted
This notice is posted in its entirety and ...
Metadata: {'doc_number': 1, 'doc_name': 'KC USACE_B_01_03_2024_AFCS_AE_Synopsis_SB_Set_Aside_W912DQ24R4019 (1) (1).pdf'}, Distance: 1.9741315841674805, Document: Synopsis Architect-Engineer (A-E) Services Indefinite Delivery/Indefinite Quantity (IDIQ) Contract F...
Metadata: {'doc_number': 4, 'doc_name': 'USACE Jacksonville SB AE.pdf'}, Distance: 1.9850943088531494, Document: An o icial website of the United States government Here’s how you know
 Follow
Contract Opportunity...
Metadata: {'doc_number': 3, 'doc_name': 'Solicitation Vicksburg.pdf'}, Distance: 2.018296003341675, Document: An o icial website of the United States government Here’s how you know
Important Reps and Certs Upda...


In [9]:
print(file_metadata)

['Doc 1: Synopsis Architect-Engineer (A-E) Services Indefinite Delivery/Indefinite Quantity (IDIQ) Contract For', 'Doc 2: Green underlined is add-in, Red strike through is deleted This', 'Doc 3: An o\x00icial website of the United States government Here’s how', 'Doc 4: An o\x00icial website of the United States government Here’s how', 'Doc 5: INDEFINITE DELIVERY CONTRACT (IDC) A-E SERVICES FOR USE WITHIN SOUTHWESTERN', 'Doc 6: C -- ARCHITECT AND ENGINEERING SERVICES W912DQ24R4002 SYNOPSIS Architect-Engineer (A-E)', 'Doc 7: Indefinite-Delivery (MATOC) for Multi-Discipline Miscellaneous Works Design and Other Architect']


In [4]:
retrived_text = " ".join(retrieved_docs)
print(retrived_text)

Green underlined is add-in, Red strike through is deleted
This notice is posted in its entirety and hereby serves as the official solicitation for this project. CONTRACT INFORMATION: A-E services will be procured in accordance with (IAW)
40 USC Chapter 11 Selection of Architects and Engineers (formerly known as Brooks
Act) and Federal Acquisition Regulation (FAR) Subpart 36.6 to provide planning,
design, and engineering during construction primarily in support of the Civil Works
mission area within the SWG boundary and may be used for similar horizontal
engineering projects to support the Military and Interagency and International Support
(IIS) missions within the SWD area of responsibility (AOR). Capacity could be
transferred outside the SWD AOR for high priority projects or emerging programs if
capacity in other regions within the continental U.S. is inadequate.
This announcement is for a Multiple Award Task Order Contract (MATOC) for A-E
services with a total shared capacity of $900

In [9]:
import requests
import json

# Initialize an empty list to store previous questions and answers
memory = []

# Pinecone text retrieval part (you've done this)
def generate_response(cx, q):
    global memory

    # Retrieved text as context from Pinecone
    retrieved_text = cx

    # Prepare the prompt with memory of past interactions
    memory_prompt = "\n".join(memory)  # Join the previous interactions
    question = q
    prompt = f"{memory_prompt}\n look at the previous answer you gave for the question {query} from this context\n{retrieved_text}\n\nbut when ever such question comes you should look up for the main purpose of the contract that means you should search for what the firmsare competiting/applying tenders or meeting there requirments.. like firms have to do some services like constrycting, modifing , demolishing etc.., to make that work there are so many firms fighting for so solicitation is a selection criteria for that..when ever contract information is asked you should tell what firms need to do after they got selected. and when contract requirements is asked then you should tell what need to be submitted while applying for that perticular contract. now tell me contract information and requirments?"

    #prompt = f"Here is some context:\n{retrieved_text}\n{memory_prompt}\n\nNow, based on this, can you answer the following question: {question}"

    # Ollama API call to generate a response
    url = "http://localhost:11434/api/generate"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3.1:latest",
        "prompt": prompt,
        "stream": False  # Disable streaming
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    # Check the response from the model
    if response.status_code == 200:
        result = response.json()
        ans = result['response']
        print("Model response:", ans)

        # Store the current question and answer in memory
        memory.append(f"Q: {question}\nA: {ans}")
        if len(memory) > 5:  
            memory = memory[-5:]

        return ans
    else:
        print("Error:", response.text)
        return None




Model response: Based on the provided information, I would say that the main moto or objective of the contract W912DQ24R4002 for Architect-Engineer (A-E) Services is:

**"To select a firm to provide architectural and engineering services for various projects in support of the Engineer District's mission, with an emphasis on teamwork, quality management, and past experience."**

This conclusion can be drawn from several points mentioned in the synopsis, such as:

* The requirement for teaming and subcontractor integration (Section D)
* The focus on Quality Management Plans (QMPs) and project-specific QMPs (Section H)
* The emphasis on recent experience and performance (Section E)
* The need to address specific selection criteria listed in Section 3, including teamwork, quality management, and past experience.


In [12]:
answer = generate_response(context, question)

Model response: Based on the provided information, the main moto or objective of the contract W912DQ24R4002 for Architect-Engineer (A-E) Services is:

"To select a firm to provide architectural and engineering services for various projects in support of the Engineer District's mission, with an emphasis on **teaming**, **quality management**, and **past experience**."


In [10]:
# Example usage:
context = retrived_text
question = query
answer = generate_response(context, question)

'what is the main moto of the contract (W912DQ24R4002) given by synopsis architect engineering A-E services'

In [4]:
answer = generate_response("did you remember your last 3 responses?", "?")

In [5]:
print(query)

notice id of solicitation W912EP22R0042. give me contract details
